In [0]:
DeltaTable.createIfNotExists(spark) \
          .tableName("default.department") \
          .addColumn("department_id", "INT") \
          .addColumn("department_name", "STRING") \
          .addColumn("department_code", "STRING") \
          .addColumn("founded_on", "STRING") \
          .execute()

Out[166]: <delta.tables.DeltaTable at 0x7f7051241460>

In [0]:
class create_staging_database:
    def create_department(self):
        DeltaTable.createIfNotExists(spark) \
          .tableName("default.department") \
          .addColumn("department_id", "INT") \
          .addColumn("department_name", "STRING") \
          .addColumn("department_code", "STRING") \
          .addColumn("founded_on", "STRING") \
          .execute()
    def create_employee(self):
        DeltaTable.createIfNotExists(spark) \
          .tableName("default.employee") \
          .addColumn("employee_id", "INT") \
          .addColumn("employee_name", "STRING") \
          .addColumn("employee_gender", "STRING") \
          .addColumn("employee_birthdate", "STRING") \
          .addColumn("employee_address", "STRING") \
          .addColumn("employee_city", "STRING") \
          .addColumn("department_id", "INT") \
          .execute()
    def create_project(self):
        DeltaTable.createIfNotExists(spark) \
          .tableName("default.project") \
          .addColumn("project_id", "INT") \
          .addColumn("project_name", "STRING") \
          .addColumn("project_code", "STRING") \
          .execute()
    def create_workon(self):
        DeltaTable.createIfNotExists(spark) \
          .tableName("default.workon") \
          .addColumn("employee_id", "INT") \
          .addColumn("project_id", "STRING") \
          .execute()

In [0]:
db = create_staging_database()
db.create_department()
db.create_employee()
db.create_project()
db.create_workon()

In [0]:
from delta.tables import *
from pyspark.sql.functions import from_unixtime

read_format_staging = 'json'
write_format_staging = 'delta'
write_mode_staging = 'overwrite'
partition_emp = 'Dep_ID'

class delta_staging:    
    def read_source_raw_data(self, load_path):
        # Load the data from its source.
        if read_format_staging == "csv":
            df = spark.read.format(read_format_staging).load(load_path)
        else:
            df = spark \
                .read \
                .option("multiLine", True) \
                .option("mode", "PERMISSIVE") \
                .json(load_path)
        # Show the results.
        display(df)
        return df

    def write_to_delta_staging(self, df, path, col, raw_type):
        # If delta table exist do Upsert (Merge) else perform Insert (Write)
        flag = DeltaTable.isDeltaTable(spark, path)
        if flag == True:
            deltaTable = DeltaTable.forPath(spark, path)
            updatesDF = df
            match = "target."+col+" = updates."+col+")"
            deltaTable.alias("target").merge(
                updatesDF.alias("updates"),
                "target."+str(col)+" = updates."+str(col) ) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll() \
              .execute()
        else:
            if raw_type == "emp_1" or df == "emp_2":
                df.write \
                   .format(write_format_staging) \
                   .partitionBy(partition_emp) \
                   .mode(write_mode_staging) \
                   .option('mergeSchema', 'true') \
                   .save(path)
            else:
                df.write \
                  .format(write_format_staging) \
                  .mode(write_mode_staging) \
                  .option('mergeSchema', 'true') \
                  .save(path)

In [0]:
# dbutils.fs.mount(
#   source = "wasbs://databrickscontainer@strgdatabricks.blob.core.windows.net",
#   mount_point = "/mnt/blobstorage",
#   extra_configs = {"fs.azure.account.key.strgdatabricks.blob.core.windows.net":"MHCAAkV6+EUbGR8dkdsnLlsR4oV+lJ6WKqEeN6sZ16OnDMeviRNBTPVj3kNuHfqpYVID44kCVnizUSux82HPXg=="})

In [0]:
load_dep_1     = 'dbfs:/mnt/blobstorage/Src_Dep_1.json'
load_dep_2     = 'dbfs:/mnt/blobstorage/Src_Dep_2.json'
load_emp_1     = 'dbfs:/mnt/blobstorage/Src_Emp_1.json'
load_emp_2     = 'dbfs:/mnt/blobstorage/Src_Emp_2.json'
load_project_1 = 'dbfs:/mnt/blobstorage/Src_Project_1.json'
load_workon_1  = 'dbfs:/mnt/blobstorage/Src_Workon_1.json'
load_workon_2  = 'dbfs:/mnt/blobstorage/Src_Workon_2.json'

delta     = delta_staging()
dep_1     = delta.read_source_raw_data(load_dep_1)
dep_2     = delta.read_source_raw_data(load_dep_2)
emp_1     = delta.read_source_raw_data(load_emp_1)
emp_2     = delta.read_source_raw_data(load_emp_2)
project_1 = delta.read_source_raw_data(load_project_1)
workon_1  = delta.read_source_raw_data(load_workon_1)
workon_2  = delta.read_source_raw_data(load_workon_2)

Dep_Code,Dep_ID,Dep_Name,Founded_On
CS,1,Computer Science,12/01/1987
IT,2,Information Technology,22/02/1992
ME,3,Mechanical,16/03/1954
CHEM,4,Chemical,23/04/1999
EL,5,Electronics,19/05/1911
MET,6,Metellurgy,03/06/1965


Dep_Code,Dep_ID,Dep_Name,Founded_On,Founded_This_Century
MNG,7,Management,13/06/2004,Yes
OPS,8,Operations,23/08/2012,Yes
FIN,9,Finance,30/11/2025,Yes
SAL,10,Sales,15/06/2017,Yes
HR,11,Human Resource,12/03/2008,Yes
PCH,12,Purchase,03/06/2019,Yes
MET_a,6,Metellurgy_new,03/06/1965,null


Address,Birthdate,City,Dep_ID,Emp_ID,Gender,Name
28 Junction St,12/01/1987,Birganbigil,1,706157,M,Prafull
15 Peterho Boulevard,10/09/1976,Yattalunga,1,454928,M,Sid
61 Farnell Street,13/03/1988,Murga,2,122405,M,Dylan
72 McGregor Street,29/08/1986,Kinalung,3,973084,F,Graham
87 Hummocky Road,13/09/1989,Karatta,4,119991,F,Zaniyah Gutierrez
94 Oriana Street,11/12/1978,Wyongah,6,119991,M,Jadiel Webster
66 Wigley Street,5/05/2017,Morphettville,3,144961,M,Claudia Conley
41 Bellion Drive,3/05/1959,Cowaramup,5,809136,F,Jaron Bonilla
26 Plug Street,1/06/1984,Ben Lomond,6,720951,M,Samara Lucero
7 Eungella Road,22/05/2000,Cannonvale,5,108537,M,Willie Watts


Address,Birthdate,City,Dep_ID,Emp_ID,Gender,Name
53 Walpole Avenue,15/02/1951,Mailer Flat,10,766391,M,Alisa Phillips
77 Tennyson Road,24/03/1966,Mount Lewis,8,210392,M,Malcolm Bartlett
55 Meyer Road,14/08/1982,Pewsey Vale,8,434780,F,Zoie Brewer
42 Adavale Road,1/10/2001,Goulburn Dc,12,459699,M,Addison Khan
1 Lewin Street,13/09/2006,Grubben,11,743630,M,Jaylin Barajas


Project_Code,Project_ID,Project_Name
FB,612,Facebook
INS,158,Instagram
GO,972,Google
ET,674,Eternals
AG,851,Avengers
RAG,507,Ragnarok
OK,205,Orbital Keys
XB,446,XPress Bottle
INP,254,InstaPress
UNW,509,Uno Wear


Emp_ID,Project_ID
706157,612
454928,158
122405,972
973084,674
119991,851
119991,507
144961,254
809136,284
720951,285
108537,539


Emp_ID,Project_ID
766391,205
210392,446
434780,509
459699,350
743630,428


In [0]:
delta.write_to_delta_staging(dep_1, '/mnt/delta/staging9/dep', "Dep_ID", "dep_1")
delta.write_to_delta_staging(emp_1, '/mnt/delta/staging9/emp', "Emp_ID", "emp_1")
delta.write_to_delta_staging(project_1, '/mnt/delta/staging9/prj', "Project_ID", "project_1")
delta.write_to_delta_staging(workon_1, '/mnt/delta/staging9/workon', "Emp_ID", "workon_1")
delta.write_to_delta_staging(dep_2, '/mnt/delta/staging9/dep', "Dep_ID", "dep_2")
delta.write_to_delta_staging(emp_2, '/mnt/delta/staging9/emp', "Emp_ID", "emp_2")
delta.write_to_delta_staging(workon_2, '/mnt/delta/staging9/workon', "Emp_ID", "workon_2")

/mnt/delta/staging9/workon
Hellooooooooooooooooooooo
Insert
/mnt/delta/staging9/dep
Test 1
Upsert
/mnt/delta/staging9/emp
Test 1
Upsert
/mnt/delta/staging9/workon
Test 1
Upsert

In [0]:
from pyspark.sql.types import IntegerType
read_format_prepared = 'json'
write_format_prepared = 'delta'
write_mode_prepared = 'overwrite'
partition_by_prepared = 'Dep_ID'

class delta_prepared:    
    def read_staging_delta_data(self, load_path):
        # Load the data from its source.
        df = spark \
          .read \
          .format("delta") \
          .load("/mnt/delta/staging8/dep")
        # Show the results.
        display(df)
        return df

    def write_to_delta_prepared(self, df, path, col, raw_type):
        
        # If delta table exist do Upsert (Merge) else perform Insert (Write)
        df.write \
          .format("delta") \
          .mode("overwrite") \
          .option('mergeSchema', 'true') \
          .saveAsTable("default.department")
    
    def modify_df_schema_dep(self, prep_df):
        prep_df = prep_df.withColumnRenamed("Dep_Code","department_code")
        prep_df = prep_df.withColumnRenamed("Dep_Id","department_id")
        prep_df = prep_df.withColumnRenamed("Dep_Name","department_name")
        prep_df = prep_df.withColumnRenamed("Founded_On","founded_on")
        prep_df = prep_df.withColumn('department_id',prep_df['department_id'].cast(IntegerType()))
        display(prep_df)
        return prep_df

In [0]:
prep = delta_prepared()
prep_dep = prep.read_staging_delta_data("/mnt/delta/staging8/dep")
prep_dep = prep.modify_df_schema_dep(prep_dep)

Dep_Code,Dep_ID,Dep_Name,Founded_On
CS,1,Computer Science,12/01/1987
IT,2,Information Technology,22/02/1992
ME,3,Mechanical,16/03/1954
CHEM,4,Chemical,23/04/1999
EL,5,Electronics,19/05/1911
MET_a,6,Metellurgy_new,03/06/1965
MNG,7,Management,13/06/2004
OPS,8,Operations,23/08/2012
FIN,9,Finance,30/11/2025
SAL,10,Sales,15/06/2017


department_code,department_id,department_name,founded_on
CS,1,Computer Science,12/01/1987
IT,2,Information Technology,22/02/1992
ME,3,Mechanical,16/03/1954
CHEM,4,Chemical,23/04/1999
EL,5,Electronics,19/05/1911
MET_a,6,Metellurgy_new,03/06/1965
MNG,7,Management,13/06/2004
OPS,8,Operations,23/08/2012
FIN,9,Finance,30/11/2025
SAL,10,Sales,15/06/2017


In [0]:
prep.write_to_delta_prepared(prep_dep, '/mnt/delta/prepared3/dep', "Dep_ID", "dep_1")

In [0]:
people = spark \
  .read \
  .format("delta") \
  .load("/mnt/delta/prepared1/dep")
display(people)

Dep_Code,Dep_ID,Dep_Name,Founded_On
CS,1,Computer Science,12/01/1987
IT,2,Information Technology,22/02/1992
ME,3,Mechanical,16/03/1954
CHEM,4,Chemical,23/04/1999
EL,5,Electronics,19/05/1911
MET_a,6,Metellurgy_new,03/06/1965
MNG,7,Management,13/06/2004
OPS,8,Operations,23/08/2012
FIN,9,Finance,30/11/2025
SAL,10,Sales,15/06/2017


In [0]:
delta.write_to_delta(dep_1, '/mnt/delta/dep', "Dep_ID")
delta.write_to_delta(emp_1, '/mnt/delta/emp', "Emp_ID")
#delta.write_to_delta(project_1, '/mnt/delta/project')
#delta.write_to_delta(workon_1, '/mnt/delta/workon')

#delta.write_to_delta(dep_2, '/mnt/delta/dep')
#delta.write_to_delta(emp_2, '/mnt/delta/emp')
#delta.write_to_delta(workon_2, '/mnt/delta/workon')

Upsert
Insert

In [0]:
read_format = 'delta'
load_path = '/mnt/delta/dep'
 
# Load the data from its source.
people = spark.read \
  .format(read_format) \
  .load(load_path)
 
# Show the results.
display(people)

Dep_ID,Dep_Name,Dep_Code
1,Computer Science,CS
2,Information Technology,IT
3,Mechanical,ME
4,Chemical,CHEM
5,Electronics,EL
6,Metellurgy,MET
10,Sales,SAL
11,Human Resource,HR
12,Purchase,PCH
7,Management,MNG


In [0]:
%pip freeze

absl-py==1.0.0
appdirs==1.4.4
argon2-cffi==20.1.0
astunparse==1.6.3
async-generator==1.10
attrs==20.3.0
backcall==0.2.0
bleach==3.3.0
boto3==1.16.7
botocore==1.19.7
cachetools==4.2.4
certifi==2020.12.5
cffi==1.14.5
chardet==4.0.0
cycler==0.10.0
Cython==0.29.23
dbus-python==1.2.16
decorator==5.0.6
defusedxml==0.7.1
distlib==0.3.2
distro==1.4.0
distro-info===0.23ubuntu1
dlt==0.2.3
entrypoints==0.3
facets-overview==1.0.0
filelock==3.0.12
flatbuffers==2.0
gast==0.4.0
google-auth==2.3.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.41.1
h5py==3.5.0
idna==2.10
ipykernel==5.3.4
ipython==7.22.0
ipython-genutils==0.2.0
ipywidgets==7.6.3
jedi==0.17.2
Jinja2==2.11.3
jmespath==0.10.0
joblib==1.0.1
jsonschema==3.2.0
jupyter-client==6.1.12
jupyter-core==4.7.1
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.0
keras==2.7.0
Keras-Preprocessing==1.1.2
kiwisolver==1.3.1
koalas==1.8.1
libclang==12.0.0
Markdown==3.3.4
MarkupSafe==1.1.1
matplotlib==3.4.2
mistune==0.8.4
nbclient==0.5.3
nbconvert==6.0.7
nbformat==5.1.3
nest-asyncio==1.5.1
notebook==6.3.0
numpy==1.19.2
oauthlib==3.1.1
opt-einsum==3.3.0
packaging==20.9
pandas==1.2.4
pandocfilters==1.4.3
parso==0.7.0
patsy==0.5.1
pexpect==4.8.0
pickleshare==0.7.5
Pillow==8.2.0
plotly==5.1.0
prometheus-client==0.10.1
prompt-toolkit==3.0.17
protobuf==3.17.2
psycopg2==2.8.5
ptyprocess==0.7.0
pyarrow==4.0.0
pyasn1==0.4.8
pyasn1-modules==0.2.8
pycparser==2.20
Pygments==2.8.1
PyGObject==3.36.0
pyparsing==2.4.7
pyrsistent==0.17.3
python-apt==2.0.0+ubuntu0.20.4.6
python-dateutil==2.8.1
pytz==2020.5
pyzmq==20.0.0
requests==2.25.1
requests-oauthlib==1.3.0
requests-unixsocket==0.2.0
rsa==4.7.2
s3transfer==0.3.7
scikit-learn==0.24.1
scipy==1.6.2
seaborn==0.11.1
Send2Trash==1.5.0
six==1.15.0
ssh-import-id==5.10
statsmodels==0.12.2
tenacity==8.0.1
tensorboard==2.7.0
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.0
tensorflow==2.7.0
tensorflow-estimator==2.7.0
tensorflow-io-gcs-filesystem==0.22.0
termcolor==1.1.0
terminado==0.9.4
testpath==0.4.4
threadpoolctl==2.1.0
tornado==6.1
traitlets==5.0.5
typing-extensions==3.10.0.2
unattended-upgrades==0.1
urllib3==1.25.11
virtualenv==20.4.1
wcwidth==0.2.5
webencodings==0.5.1
Werkzeug==2.0.2
widgetsnbextension==3.5.1
wrapt==1.13.3